# Export Landsat Images through GEE API

This code is used to export the JRC/GSW1_4/MonthlyHistory images based on predefined rectangles of the Congo and Indus rier systems. More information of the JRC-GSWD data can be found in ...

**Background:**
- This script provides an optimized workflow from the original code by (Margerini., 2023). Instead of manually running each task in Google Earth Engine (GEE), Google Colab combined with the GEE API allows for automatic task excecution.
- Read more about this work in 3.1.1 from (Margherini., 2023)

## Preliminaries

In order to succesfully run this script the following tasks have to be done beforehand:
- Create a google earth engine project to work in.
- Create your own rectangles to load into GEE.
  - This project

In [1]:
import ee
import datetime
import time
from tqdm import tqdm
import json

# Shell command in case GEE authentication is broken !ONLY RUN WHEN 404 AUTHENTICATION ERROR PERSITS!
#!rm -rf ~/.config/earthengine

# Initialize Google Earth Engine
ee.Authenticate(force=True)
ee.Initialize(project='dsaie-481410')

In [2]:
# Mount google driwe
from google.colab import drive
drive.mount('/content/drive')

# Define project path in google driwe
#proj_path = '../My Drive/TU Delft/DS and AI for Engineers/MORPH_project'        # this does not work in GEE

Mounted at /content/drive


In [3]:
# Define training rectangles
indus_training_rectangles = ee.FeatureCollection('projects/dsaie-481410/assets/Indus_training_rectangles_fixed')
ghangara_training_rectangles = ee.FeatureCollection('projects/dsaie-481410/assets/Ghangara_training_rectangles')
ganges_training_rectangles = ee.FeatureCollection('projects/dsaie-481410/assets/Ganges_training_rectangles')

# Create combined object
combined = ghangara_training_rectangles.merge(indus_training_rectangles)
combined = combined.merge(ganges_training_rectangles)

# import geemap
import geemap

# add map and rectangles
map = geemap.Map()
map.add_basemap('HYBRID')

# Add center object
map.centerObject(combined, zoom = 3)

# Add training rectangles
map.addLayer(ghangara_training_rectangles, {}, 'Ghangara training Rectangles')
map.addLayer(indus_training_rectangles, {}, 'Indus training Rectangles')
map.addLayer(ganges_training_rectangles, {}, 'Ganges training Rectangles')

# Display map
map

Map(center=[27.825548403396354, 79.45554870534724], controls=(WidgetControl(options=['position', 'transparent_…

## Load in the rectangles and extract the landsat-5 data

In the original research paper by Antonio Margherini the *Monthly History GSWD* from **link** was used. This one will also be used to export the Congo images.

In [ ]:
# Collect rectangle metadata for all rectangles
rectangles_info = ganges_training_rectangles.getInfo()['features']

print(f"Number of rectangles = {len(rectangles_info)}")

#rectangles_info

Number of rectangles = 12


### User controls

The following block allows you to modify the eentual output of the script. Changeable parameters:

- **Collection:** Define JRC collection. More information on different collection can be found here ...

- **Period:** Load in the years which you want to be represented in the collection, f.i. GSW1_4 spans from 2984 - 2021.

- **Years to run:** Define the years of which you want the eentual images.

- **Months to run:** Define of whihc months you wat the image. This parameter in crucial -due to seasonal cloud coerage- especially in areas near the equator.

- **basin:** Of which rier/basin are you extracting the information? (important for output folder).

- **ID's of rectangles:** Define which rectangles you want to image. *Recommended to start with only ONE id to make sure the script works.*

- **Throttling:** This parameter is crucial to make sure the GEE tasks manager doesn't oerflow. recommended to stay under 280 simultaneous tasks altought the default in this script is 100. The second ariable *SLEEP_SECONDS* defines how many seconds the script should *pause* before continuing. In this script it is assumed that each GEE exporting task (or exporting a singular image) takes around 2 minutes.

- **Water palette:** *(optional)* define the colors the three outputs in each tiff should take on. This should **only** be done for isualization purposes. When working with the images in a CNN, remoe this line!

- **Scale:** The resolution of the pixels in the image in meters.

In [ ]:
# --- USER CONTROLS ---

# Define collection
collection = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory')

# Filter data range and collection
start_period = ee.Date('1983-12-01')      # first year: ('1987-12-01')
end_period = ee.Date('2022-01-01')        # last year: 2021
collection = collection.filterDate(start_period, end_period)

print(f"Start = {start_period.getInfo()}")
print(f"End = {end_period.getInfo()}")
print(f"Number of images = {collection.size().getInfo()}")

# Build a list of years based on start and end dates
start_year = 1999 # first full year after Dec 1987
end_year = 2021    # update according to dataset end
years_to_run = list(range(start_year, end_year + 1))

# Define which month(s) to run
months_to_run = [1,2,3]

# Define rier
basin = 'Ganges'

# Rectangles to process (indices into rectangles_info)
# rect_indices = [0, 3, 5]    # specific rectangles --OR--
rect_ids = range(1,len(rectangles_info)+1)  # all rectangles
print(f"Rectangle range = {rect_ids}")


# Throttling
TASKS_PER_BATCH = 1000
SLEEP_SECONDS = 60 * TASKS_PER_BATCH  # 1 minute per run

# Define water palette
water_pallete = ['white','green','blue']

# define resolution of pixels in image
scale = 60    # in meters

print('\n')
print(f"Task length = {len(rect_ids) * len(months_to_run) * len(years_to_run)}")

Start = {'type': 'Date', 'value': 439084800000}
End = {'type': 'Date', 'value': 1640995200000}
Number of images = 454
Rectangle range = range(1, 13)


Task length = 828


### Excecution block

In [ ]:
submitted_tasks = []

# Reload already submitted tasks
try:
  with open('Submitted_tasks.json', 'r') as f:
    submitted_tasks = json.load(f)

except FileNotFoundError:
  submitted_tasks = []

# Create set of already submitted tasks
submitted_set = set([(t['rect_id'], t['year'], t['month']) for t in submitted_tasks])

# Count already submitted sets
task_counter = len(submitted_set)

print(f"Resuming tasks after {task_counter} preiously submitted tasks.")

# Build lookup dictionary: id -> rectangle feature
rectangles_by_id = {
    rect['properties']['id']: rect
    for rect in rectangles_info
}

for rect_id in rect_ids:

    if rect_id not in rectangles_by_id:
        print(f"WARNING: Rectangle ID {rect_id} not found. Skipping.")
        continue

    rect = rectangles_by_id[rect_id]
    geometry = ee.Geometry.Polygon(rect['geometry']['coordinates'])

    print(f'\nProcessing rectangle {rect_id}')

    for y in years_to_run:
      for month in months_to_run:

          # Cehck if task has already been submitted
          if (rect_id, y, month) in submitted_set:
            continue

          # ---- HARD THROTTLE ----
          if task_counter > 0 and task_counter % TASKS_PER_BATCH == 0:
              print(f'\nSubmitted {task_counter} tasks. Sleeping for {SLEEP_SECONDS/60} minutes...\n')
              time.sleep(SLEEP_SECONDS)

          # Define month window
          start = ee.Date.fromYMD(y, month, 1)
          end = start.advance(1, 'month')

          # Month collection
          month_collection = collection.filterDate(start, end)

          # Minimal check (keep as in your original fast script)
          if month_collection.size().getInfo() == 0:
              print(f"No images for {y}-{month:02d}")
              continue

          # Mosaic + clip
          month_image = month_collection.mosaic().clip(geometry)

          # Visualize
          month_image_is = month_image # .visualize(
          #     min=0,
          #     max=2,
          #     palette=water_pallete
          # )

          # Naming
          month_str = f'{month:02d}'
          export_name = f'JRC_GSW1_4_Monthlyhistory_{basin}_{y}-{month_str}_r{rect_id}'
          folder_name = f'JRC_GSW1_4Monthlyhistory_{basin}_{month_str}_r{rect_id}'

          # Export
          ee.batch.Export.image.toDrive(
              image=month_image,
              description=export_name,
              folder=folder_name,
              region=geometry,
              scale=scale,
              maxPixels=1e13
          ).start()

          task_counter += 1
          print(f'[{task_counter}] Export task started: {export_name}')

          # Sae task in list
          submitted_tasks.append({
              'rect_id': rect_id,
              'year': y,
              'month': month,
          })
          submitted_set.add((rect_id, y, month))

          # Sae task in json direc
          if task_counter % 10 ==0:
            with open('Submitted_tasks.json', 'w') as f:
              json.dump(submitted_tasks, f, indent=2)

Resuming tasks after 0 preiously submitted tasks.

Processing rectangle 1
[1] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_1999-01_r1
[2] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_1999-02_r1
[3] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_1999-03_r1
[4] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2000-01_r1
[5] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2000-02_r1
[6] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2000-03_r1
[7] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2001-01_r1
[8] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2001-02_r1
[9] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2001-03_r1
[10] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2002-01_r1
[11] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2002-02_r1
[12] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2002-03_r1
[13] Export task started: JRC_GSW1_4_Monthlyhistory_Ganges_2003-01_r1
[14] Export task started: